In [ ]:
import requests
from itertools import batched
from pathlib import Path
import shutil

COOK_COUNTY_GIS_URL = (
    "https://gis.cookcountyil.gov/imagery/rest/services/CookOrtho2023/ImageServer"
)

# Cook County Multispectral Imagery

Tiles index:
> https://hub-cookcountyil.opendata.arcgis.com/datasets/cookcountyil::aerial-imagery-reference-tiles/explore

Query Service:
> https://gis.cookcountyil.gov/imagery/rest/services/CookOrtho2023/ImageServer/query

Download:
> https://gis.cookcountyil.gov/imagery/rest/services/CookOrtho2023/ImageServer/download

In [ ]:
def get_tile_id(tile_number: list[str]):
    # SQL Query
    str_tiles = ",".join([f"'{s}'" for s in tile_number])
    sql_query = f"Name IN ({str_tiles})"
    
    # REST API endpoint
    url = f"{COOK_COUNTY_GIS_URL}/query"

    # Adding parameters to query
    payload = dict(
        where=sql_query,
        returnIdsOnly=True,
        returnGeometry=False,
        f="pjson",
    )

    # A get request to the API
    response = requests.get(url, params=payload)
    response.raise_for_status()
    return response.json()

In [ ]:
get_tile_id(('16507850', '15507900',))

In [ ]:
def get_tif_download_urls(tile_id):

    # REST API endpoint
    url = url = f"{COOK_COUNTY_GIS_URL}/download"

    # Adding parameters to query
    payload = dict(rasterIds=tile_id, format="tif", f="pjson")

    # A get request to the API
    response = requests.get(url, params=payload)
    response.raise_for_status()
    return response.json()

In [ ]:
get_tif_download_urls(4212)

In [ ]:
WORKING_DIR = Path("Edwin")
PATH_LIST_TILES = WORKING_DIR/"tile_list"
FINAL_DEST = Path("/media/edwin/LaCie/Cook County Multispectral/2023")

In [ ]:
path_list_ids = WORKING_DIR/"tile_ids"
(download_dir := WORKING_DIR/"download").mkdir(exist_ok=True)

In [ ]:
with open(PATH_LIST_TILES) as f:
    tile_list = list(set([x.strip() for x in f.readlines()]))

In [ ]:
tile_ids = []

for tile_chunck in batched(tile_list, 20):
    response = get_tile_id(tile_chunck)
    tile_ids += response['objectIds']

In [ ]:
# Write to a file the list of tiles ids
with open(path_list_ids, "w") as f:
    for tile_id in tile_ids:
        f.write(str(tile_id) + "\n")

In [ ]:
for tile_id in tile_ids:
    for raster_json in get_tif_download_urls(tile_id)['rasterFiles']:
        raster_url = raster_json['id']
        raster_file = download_dir/(raster_url.rpartition('\\')[-1])
        raster_id = raster_json['rasterIds'][0]
        
        # /file?id=.%5C2023%5CRGBNIR%5C14507850.tif&amp;rasterId=3444

        if (FINAL_DEST/raster_file.name).exists():
            print(f"Skipping {raster_file.name}, already downloaded")
    
        else:
            print(f"Downloading {raster_file}")

            payload = dict(
                id=raster_url,
                rasterId=raster_id,
            )

            r = requests.get(f"{COOK_COUNTY_GIS_URL}/file?", params=payload, stream=True)
            r.raise_for_status()
        
            with open(raster_file, 'wb') as f:
                f.write(r.content)

    
            shutil.move(raster_file, FINAL_DEST)
            print(f"Downloaded {raster_file.name}")